# **[HW3] VGG Network**
1. Convolution Operation
2. VGG Network
3. Train CIFAR-10 with own VGG
4. Train CIFAR-10 with pre-trained VGG

이번 실습에서는 Image분류에서 종종 baseline model로 사용되는 VGG-Network를 직접 구현해보는 시간을 갖도록 하겠습니다.



## Import packages

런타임의 유형을 변경해줍니다.

상단 메뉴에서 [런타임]->[런타임유형변경]->[하드웨어가속기]->[GPU]

변경 이후 아래의 cell을 실행 시켰을 때, torch.cuda.is_avialable()이 True가 나와야 합니다.



In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils import data
print(torch.__version__)
print(torch.cuda.is_available())

1.12.1+cu113
False


In [28]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import tqdm
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

# 1. Convolution Operation

PyTorch에서는 [nn.sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html#torch.nn.Sequential) 등의 함수를 이용해서 복잡한 모델 구조를 종종 축약해서 사용하곤 합니다.

아래의 예제는 conv-relu-maxpool의 model을 서로 다른 방법으로 표현한 것입니다.

In [29]:
input_image = torch.rand(64, 3, 32, 32)
input_image.shape

torch.Size([64, 3, 32, 32])

### nn.Sequential

In [30]:
class Conv1(nn.Module):
    def __init__(self): # input image = batch_size x 3 x 32 x 32
        super(Conv1, self).__init__()
        self.conv = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.maxpool(x)
        return x


class Conv2(nn.Module):
    def __init__(self): # input image = batch_size x 3 x 32 x 32
        super(Conv2, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))

    def forward(self, x):
        x = self.layer(x)
        return x


class Conv3(nn.Module):
    def __init__(self): # input image = batch_size x 3 x 32 x 32
        super(Conv3, self).__init__()
        layer = []
        layer.append(nn.Conv2d(3, 64, kernel_size=3, padding=1))
        layer.append(nn.ReLU())
        layer.append(nn.MaxPool2d(2))
        self.layer = nn.Sequential(*layer)

    def forward(self, x):
        x = self.layer(x)
        return x

model1 = Conv1()
model2 = Conv2()
model3 = Conv3()

In [31]:
print(model1)
output = model1(input_image)
print(output.size())

Conv1(
  (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
torch.Size([64, 64, 16, 16])


In [32]:
print(model2)
output = model2(input_image)
print(output.size())

Conv2(
  (layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)
torch.Size([64, 64, 16, 16])


In [33]:
print(model3)
output = model3(input_image)
print(output.size())

Conv3(
  (layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)
torch.Size([64, 64, 16, 16])


### Let's practice to calculate the shape of the network

In [34]:
class Conv(nn.Module):
    def __init__(self): # input image = batch_size x 3 x 32 x 32
        super(Conv, self).__init__()
        self.conv1 = nn.Conv2d(3, 512, 3, 1, 1) 
        self.conv2 = nn.Conv2d(512, 256, 3, 1, 1) 
        self.conv3 = nn.Conv2d(256, 256, 3, 2, 1) 
        ## linear layer의 shape에 맞게 convolution layer의 configuration을 setting하세요. 
        ## YOUR CODE HERE (~ 1 line)
        # inchannel outchannel kernelsize stride padding 16 by 16 to 4 by 4?
        self.conv4 = nn.Conv2d(in_channels=256,out_channels=256,kernel_size=5,stride=4,padding = 1)
        ## END OF YOUR CODE
        self.linear = nn.Linear(256*4*4, 10)

    def forward(self, x):
        out=self.conv1(x)
        out=self.conv2(out)
        out=self.conv3(out)
        out=self.conv4(out)
        out = out.contiguous().view(-1, 256*4*4)
        out = self.linear(out)
        return out

In [35]:
model = Conv()
print(model)
output = model(input_image)
print(output.size())

Conv(
  (conv1): Conv2d(3, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(256, 256, kernel_size=(5, 5), stride=(4, 4), padding=(1, 1))
  (linear): Linear(in_features=4096, out_features=10, bias=True)
)
torch.Size([64, 10])


# 2. VGG Network

이번 section에서는 3*3의 filter로 이루어진 19-layer VGG-network를 직접 구현해보도록 하겠습니다.

자세한 모델의 configuration은 아래 그림의 option E와 같습니다.

![](https://drive.google.com/uc?export=view&id=1mFwqxP5rB4lhEfRk7OJla1wffKqEibQy)


VGG-19는 2layer로 구성된 2개의 convolution block과 4 layer로 구성된 3개의 convolution block으로 나누어져있습니다.

또한, 학습 안정성을 위하여 각 layer는 convolution-batchnorm-relu로 이루어지게 되며 매 block에 끝마다 2*2 max pooling을 진행해주게 됩니다. 



In [36]:
class TwoLayerBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TwoLayerBlock, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        ## 위의 model configuration을 참조하여 two-layer block을 구현하세요. 
        ## YOUR CODE HERE (~ 7 lines)
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=input_dim,out_channels=output_dim,kernel_size=3,stride=1,padding = 1),
            nn.BatchNorm2d(output_dim),
            nn.ReLU(),
            nn.Conv2d(in_channels=output_dim,out_channels=output_dim,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(output_dim),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        ## END OF YOUR CODE
        
    def forward(self, x):
        x = self.layer(x)
        return x

In [37]:
class FourLayerBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(FourLayerBlock, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        ## 위의 model configuration을 참조하여 four-layer block을 구현하세요. 
        ## YOUR CODE HERE (~ 13 lines)
        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=input_dim,out_channels=output_dim,kernel_size=3,stride=1,padding = 1),
            nn.BatchNorm2d(output_dim),
            nn.ReLU(),
            nn.Conv2d(in_channels=output_dim,out_channels=output_dim,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(output_dim),
            nn.ReLU(),
            nn.Conv2d(in_channels=output_dim,out_channels=output_dim,kernel_size=3,stride=1,padding = 1),
            nn.BatchNorm2d(output_dim),
            nn.ReLU(),
            nn.Conv2d(in_channels=output_dim,out_channels=output_dim,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(output_dim),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        ## END OF YOUR CODE
        
    def forward(self, x):
        x = self.layer(x)
        return x

In [38]:
class VGG19(nn.Module):
    def __init__(self):
        super(VGG19, self).__init__()
        
        self.block1 = TwoLayerBlock(3, 64)
        self.block2 = TwoLayerBlock(64, 128)
        self.block3 = FourLayerBlock(128, 256)
        self.block4 = FourLayerBlock(256, 512)
        self.block5 = FourLayerBlock(512, 512)
        self.linear = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = x.squeeze() 
        x = self.linear(x)
        return x

In [39]:
model = VGG19()
print(model)

output = model(input_image)
print(output.size())

VGG19(
  (block1): TwoLayerBlock(
    (layer): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (block2): TwoLayerBlock(
    (layer): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): MaxPool2d(kernel_size=2, stride=2,

### Check Implementations

In [40]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
if count_parameters(model) == 20365002:
    print('success!')
else:
  raise AssertionError

success!


# 3. Train CIFAR-10 with own VGG

train-image: 50,000
test-image: 10,000

class: [비행기, 자동차, 트럭, 개구리, ...] 등 10개의 class

for more info, https://www.cs.toronto.edu/~kriz/cifar.html

### Use pre-defined dataset

PyTorch는 custom dataset과 dataloader를 사용해도 되지만 cifar-10과 같은 유명 데이터셋에 대해서는 pre-defined된 dataset이 존재합니다.

이번 실습에서는 custom dataset을 직접 만드는 대신 pre-trained dataset을 불러와서 실습을 진행해보도록 하겠습니다.

In [41]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

train_dataset = datasets.CIFAR10(root='./data/',      
                                 train=True, 
                                 transform=transforms.ToTensor(),
                                 download=True)

test_dataset = datasets.CIFAR10(root='./data/',
                                train=False, 
                                transform=transforms.ToTensor())

batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data/


In [42]:
class Trainer():
    def __init__(self, trainloader, testloader, model, optimizer, criterion, device):
        """
        trainloader: train data's loader
        testloader: test data's loader
        model: model to train
        optimizer: optimizer to update your model
        criterion: loss function
        """
        self.trainloader = trainloader
        self.testloader = testloader
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device
        
    def train(self, epoch = 1):
        self.model.train()
        loss_list = []
        acc_list = []
        for e in range(epoch):
            running_loss, running_acc = 0.0, 0.0
            for i, data in tqdm.tqdm(enumerate(self.trainloader, 0)): 
                inputs, labels = data 
                # model에 input으로 tensor를 gpu-device로 보낸다
                inputs = inputs.to(self.device)  
                labels = labels.to(self.device)
                # zero the parameter gradients
                self.optimizer.zero_grad()    
                # forward + backward + optimize
                outputs = self.model(inputs) 
                loss = self.criterion(outputs, labels)  
                loss.backward() 
                self.optimizer.step() 
                running_loss += loss.item()
                pred = outputs.max(1, keepdim=True)[1]
                running_acc += pred.eq(labels.view_as(pred)).sum().item()
            
            running_loss = running_loss / len(self.trainloader)
            running_acc = running_acc / len(self.trainloader.dataset)
            loss_list.append(running_loss)
            acc_list.append(running_acc)
            print('epoch: %d  loss: %.3f  acc:%.3f' % (e + 1, running_loss, running_acc))
            
        return loss_list, acc_list

    def test(self):
        self.model.eval() 
        correct = 0
        for inputs, labels in self.testloader:
            inputs = inputs.to(self.device)
            labels = labels.to(self.device)
            output = self.model(inputs) 
            pred = output.max(1, keepdim=True)[1] # get the index of the max 
            correct += pred.eq(labels.view_as(pred)).sum().item()
        test_acc = correct / len(self.testloader.dataset)
        print('test_acc: %.3f' %(test_acc))

In [43]:
model = VGG19()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
#device = torch.device('cuda')
device = torch.device('cpu')
model.to(device)

VGG19(
  (block1): TwoLayerBlock(
    (layer): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (block2): TwoLayerBlock(
    (layer): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): MaxPool2d(kernel_size=2, stride=2,

In [45]:
trainer = Trainer(trainloader = train_loader,
                  testloader = test_loader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

loss_list, acc_list = trainer.train(epoch = 10)

0it [00:01, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
x = np.arange(10)

fig, axs = plt.subplots(1, 2)
axs[0].plot(x, loss_list)
axs[0].set_xlabel('epoch')
axs[0].set_title('loss_plot')

axs[1].plot(x, acc_list)
axs[1].set_xlabel('epoch')
axs[1].set_title('acc_plot')

plt.tight_layout()
plt.show()

In [ ]:
trainer.test()

### Question) 왜 test-accuracy가 train-accuracy에 비해 현저히 낮은 성능을 보일까요?

ans) 오버피팅 때문입니다. 

# 4. Train CIFAR-10 with pre-trained VGG

비록 저희가 직접 학습한 VGG-network를 통해 reasonable한 accuracy를 달성할 수 있었지만, 더 큰 모델을 사용할 경우 매번 처음부터 학습을 진행하는 것은 쉬운 일이 아닙니다. 

따라서, 이번 섹션에서는 pre-trained된 VGG model을 불러와서 linear layer만 추가적으로 학습시켜 적은 시간의 학습 만으로도 높은 성능을 달성해보는 실습을 진행해보겠습니다.

PyTorch에서 공식적으로 제공하는 pre-trained model은 https://pytorch.org/vision/stable/models.html 해당 document에서 찾아볼 수 있으며, 종종 google이나 facebook등에서는 자신들이 개발한 모델에 대해서 pre-trained된 weight를 https://github.com/google-research/simclr 이와 같이 배포하고는 합니다.

In [46]:
import torchvision.models as models
pretrained_vgg = models.vgg19_bn(pretrained=True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [47]:
class Pretrained_VGG19(nn.Module):
    def __init__(self, pretrained_model):
        super(Pretrained_VGG19, self).__init__()
        # inherit the weights from the pre-trained model
        self.features = nn.Sequential(
            *list(pretrained_model.features.children())
        )
        self.linear = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )
    def forward(self, x):
        x = self.features(x)
        x = x.squeeze() 
        x = self.linear(x)
        return x

In [48]:
model = Pretrained_VGG19(pretrained_vgg)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
#device = torch.device('cuda')
device = torch.device('cpu')
model.to(device)

Pretrained_VGG19(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Co

In [49]:
trainer = Trainer(trainloader = train_loader,
                  testloader = test_loader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

loss_list, acc_list = trainer.train(epoch = 1)

1it [00:07,  7.65s/it]


KeyboardInterrupt: ignored

In [ ]:
trainer.test()